# Classes

In [82]:
import random

class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"
    
class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <=2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
        
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]

    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]

    
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)


# Load Data

In [83]:
import json

#file_name = 'books_small.json'

file_name = 'books_small_10000.json'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))
        
        
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

# Prep Data

In [84]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_cont = ReviewContainer(train)
test_cont = ReviewContainer(test)

In [85]:
train_cont.evenly_distribute()
test_cont.evenly_distribute()

train_x = train_cont.get_text()
train_y = train_cont.get_sentiment()

test_x = test_cont.get_text()
test_y = test_cont.get_sentiment()


##### Bag of words

In [86]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()

train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)


# Classification

#### SVM

In [87]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')

clf.fit(train_x_vectors, train_y)

svm_score = (clf.score(test_x_vectors, test_y))

print('SVM Score:', svm_score)


SVM Score: 0.8076923076923077


#### Decision Tree

In [88]:
from sklearn.tree import DecisionTreeClassifier

dec = DecisionTreeClassifier()

dec.fit(train_x_vectors, train_y)

dec_score = dec.score(test_x_vectors, test_y)

print('Decision Tree Score:', dec_score)

Decision Tree Score: 0.6370192307692307


#### Naive Bayes

In [89]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(train_x_vectors.toarray(), train_y)

nb_score = nb.score(test_x_vectors.toarray(), test_y)

print('Naive Bayes Score:', nb_score)

Naive Bayes Score: 0.6610576923076923


#### Logistic Regression

In [90]:
from sklearn.linear_model import LogisticRegression

log = LogisticRegression()

log.fit(train_x_vectors, train_y)

log_score = log.score(test_x_vectors, test_y)

print('Decision Tree Score:', log_score)

Decision Tree Score: 0.8052884615384616


# F1 Scores

In [91]:
from sklearn.metrics import f1_score

print(f1_score(test_y, clf.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, dec.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, nb.predict(test_x_vectors.toarray()), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))
print(f1_score(test_y, log.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE]))


[0.80582524 0.80952381]
[0.63080685 0.643026  ]
[0.65693431 0.66508314]
[0.80291971 0.80760095]


# Manual Testing

In [92]:
test_set = ['not quite good!', 'I loved it', 'time waste']
new_test = vectorizer.transform(test_set)

clf.predict(new_test)

array(['NEGATIVE', 'POSITIVE', 'NEGATIVE'], dtype='<U8')

In [96]:
test_set = ['not quite good!', 'I loved it', 'time waste']
new_test = vectorizer.transform(test_set)

log.predict(new_test)

array(['NEGATIVE', 'POSITIVE', 'NEGATIVE'], dtype='<U8')

# Grid Search

In [98]:
from sklearn.model_selection import GridSearchCV

params = {'kernel': ('linear', 'rbf'), 'C': (1,2,4,8,16,32)}

tuned_svc = SVC()

tuned_clf = GridSearchCV(tuned_svc, params, cv=5)
tuned_clf.fit(train_x_vectors, train_y)

tuned_svm_score = (tuned_clf.score(test_x_vectors, test_y))

print('Tuned SVM Score:', tuned_svm_score)
# It increased 0.01!

Tuned SVM Score: 0.8173076923076923


# Saving Model

In [100]:
import pickle

with open('classifier.pkl', 'wb') as f:
    pickle.dump(tuned_clf, f)

# Load Model

In [101]:
with open('classifier.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)

In [103]:
loaded_clf.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')